In [ ]:
# import relevant libraries
%matplotlib inline

import json
import os
import sys

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Rectangle, ConnectionPatch
from matplotlib.offsetbox import  OffsetImage
from matplotlib import transforms
import squarify


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tscraper.tscraper import get_complete_data

In [ ]:
def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

def draw_pitch(ax, color="lightgray"):
    # size of the pitch is 120, 80
    #Create figure

    #Pitch Outline & Centre Line
    plt.plot([0,0],[0,80], color=color)
    plt.plot([0,120],[80,80], color=color)
    plt.plot([120,120],[80,0], color=color)
    plt.plot([120,0],[0,0], color=color)
    plt.plot([60,60],[10,70], color=color)

    #Left Penalty Area
    plt.plot([14.6,14.6],[57.8,22.2],color=color)
    plt.plot([0,14.6],[57.8,57.8],color=color)
    plt.plot([0,14.6],[22.2,22.2],color=color)

    #Right Penalty Area
    plt.plot([120,105.4],[57.8,57.8],color=color)
    plt.plot([105.4,105.4],[57.8,22.5],color=color)
    plt.plot([120, 105.4],[22.5,22.5],color=color)

    #Left 6-yard Box
    plt.plot([0,4.9],[48,48],color=color)
    plt.plot([4.9,4.9],[48,32],color=color)
    plt.plot([0,4.9],[32,32],color=color)

    #Right 6-yard Box
    plt.plot([120,115.1],[48,48],color=color)
    plt.plot([115.1,115.1],[48,32],color=color)
    plt.plot([120,115.1],[32,32],color=color)

    #Prepare Circles
    centreCircle = plt.Circle((60,40),8.1,color=color,fill=False)
    centreSpot = plt.Circle((60,40),0.71,color=color)
    leftPenSpot = plt.Circle((9.7,40),0.71,color=color)
    rightPenSpot = plt.Circle((110.3,40),0.71,color=color)

    #Draw Circles
    ax.add_patch(centreCircle)
    ax.add_patch(centreSpot)
    ax.add_patch(leftPenSpot)
    ax.add_patch(rightPenSpot)

    #Prepare Arcs
    # arguments for arc
    # x, y coordinate of centerpoint of arc
    # width, height as arc might not be circle, but oval
    # angle: degree of rotation of the shape, anti-clockwise
    # theta1, theta2, start and end location of arc in degree
    leftArc = Arc((9.7,40),height=16.2,width=16.2,angle=0,theta1=310,theta2=50,color=color)
    rightArc = Arc((110.3,40),height=16.2,width=16.2,angle=0,theta1=130,theta2=230,color=color)

    #Draw Arcs
    ax.add_patch(leftArc)
    ax.add_patch(rightArc)
    
    #Referee Area
    plt.plot([45,75],[10,10],color=color)
    plt.plot([45,45],[0.2,10],color=color)
    plt.plot([75,75],[0.2,10],color=color)
    #Coach Area
    plt.plot([45,75],[70,70],color=color)
    plt.plot([45,45],[70,79.8],color=color)
    plt.plot([75,75],[70,79.8],color=color)
    
    
def rainbow_text(x, y, strings, font_colors, font_weights=None, font_sizes=None,
                 orientation='horizontal', ax=None, sep="\n", **kwargs):
    """
    Take a list of *strings* and *colors* and place them next to each
    other, with text strings[i] being shown in colors[i].

    Parameters
    ----------
    x, y : float
        Text position in data coordinates.
    strings : list of str
        The strings to draw.
    colors : list of color
        The colors to use.
    orientation : {'horizontal', 'vertical'}
    ax : Axes, optional
        The Axes to draw into. If None, the current axes will be used.
    **kwargs
        All other keyword arguments are passed to plt.text(), so you can
        set the font size, family, etc.
    """
    if ax is None:
        ax = plt.gca()
    t = ax.transAxes
    canvas = ax.figure.canvas

    assert orientation in ['horizontal', 'vertical']
    if orientation == 'vertical':
        kwargs.update(rotation=90, verticalalignment='bottom')

    for idx, (s, c) in enumerate(zip(strings, font_colors)):
        if font_weights:
            weight = font_weights[idx]
            if font_sizes:
                size = font_sizes[idx]
                text = ax.text(x, y, s + sep, fontweight=weight, fontsize=size, color=c, transform=t, **kwargs)
            else:
                text = ax.text(x, y, s + sep, fontweight=weight, color=c, transform=t, **kwargs)
        elif font_sizes:
            size = font_sizes[idx]
            text = ax.text(x, y, s + sep, fontsize=size, color=c, transform=t, **kwargs)
        else:
            text = ax.text(x, y, s + sep, color=c, transform=t, **kwargs)

        # Need to draw to update the text position.
        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()
        if orientation == 'horizontal' and sep == "\n":
            t = transforms.offset_copy(text.get_transform(), y=-ex.height/1.5, units='dots')
        elif orientation == 'horizontal':
            t = transforms.offset_copy(text.get_transform(), x=ex.width, units='dots')
        else:
            t = transforms.offset_copy(text.get_transform(), y=ex.height, units='dots')

In [ ]:
def print_bench(df, team, ax, pos='left', comp=None):
    team_name = team[0][0]['team']
    df_team = df[(df['is_bench'] == True) & (df['team'] == team_name)]
    for idx, (_, player) in enumerate(df_team.iterrows()):
        age_data = ' | '.join([player['age'].split(' ')[0], player['birth_date'].split(' ')[-1], player['country'].capitalize()[:3]])
        data = [str(player['number']).strip(), player['full_name'].strip(), player['position'].strip(), age_data.strip()]
        font_colors = ["red", "black", "black", "blue"]
        font_sizes = [14, 14, 14, 14]
        font_weights = ['extra bold', 'black', 'black', 'black']
        coord_x = 0.0
        if pos == 'right':
            coord_x = 0.5
        coord_y = 1 - idx / len(df_team)
        rainbow_text(coord_x, coord_y, sep=' || ', strings=data, font_colors=font_colors, ax=ax, font_sizes=font_sizes, font_weights=font_weights)

        nat_pres = player['nat_presences']
        nat_goals = player['nat_goals']
        nat_assists = player['nat_assists']
        season_pres = 0
        season_goals = 0
        season_assists = 0
        for key, value in player['year_summary_data'].items():
            if value[0] != '-':
                season_pres += int(value[0])
            if value[1] != '-':
                season_goals += int(value[1])
            if value[2] != '-':
                season_assists += int(value[2])
        nat_data = 'NAT: ' + ' | '.join([str(elem) for elem in [nat_pres, nat_goals, nat_assists]])
        season_data = 'SEA: ' + ' | '.join([str(elem) for elem in [season_pres, season_goals, season_assists]])
        data = [nat_data, season_data]
        font_colors = ["blue", "green", "blue"]
        font_sizes = [12, 12]
        font_weights = ['normal', 'normal', 'normal']
        if comp and comp in player['year_summary_data']:
            comp_data = player['year_summary_data'][comp]
            comp_str = 'COMP: ' + ' | '.join([str(comp_data[0]), str(comp_data[1]), str(comp_data[2])])
            data.append(comp_str)
            font_colors.append("green")
            font_sizes.append(12)
            font_weights.append("normal")
        coord_x = 0.05
        if pos == 'right':
            coord_x = 0.55
        coord_y -= 0.025
        rainbow_text(coord_x, coord_y, sep=' ---- ', strings=data, font_colors=font_colors, ax=ax, font_sizes=font_sizes, font_weights=font_weights)

In [ ]:
def print_team(df, team, ax, pos='left', coach=None, comp=None):
    total_lines = len(team)
    for x_idx, line in enumerate(team):
        players_in_line = len(line)
        for y_idx, player in enumerate(line):
            nat_pres = player['nat_presences']
            nat_goals = player['nat_goals']
            nat_assists = player['nat_assists']
            season_pres = 0
            season_goals = 0
            season_assists = 0
            for key, value in player['year_summary_data'].items():
                if value[0] != '-':
                    season_pres += int(value[0])
                if value[1] != '-':
                    season_goals += int(value[1])
                if value[2] != '-':
                    season_assists += int(value[2])
            age_data = ' | '.join([player['age'].split(' ')[0], player['birth_date'].split(' ')[-1], player['country'].capitalize()[:3]])
            nat_data = ' | '.join([str(elem) for elem in [nat_pres, nat_goals, nat_assists]])
            season_data = ' | '.join([str(elem) for elem in [season_pres, season_goals, season_assists]])
            data = [str(player['number']), player['full_name'], age_data, nat_data, season_data]
            font_colors = ["red", "black", "blue", "green", "blue"]
            font_sizes = [14, 14, 14, 12, 12]
            font_weights = ['extra bold', 'black', 'black', 'normal', 'normal']
            if comp and comp in player['year_summary_data']:
                comp_data = player['year_summary_data'][comp]
                comp_str = ' | '.join([str(comp_data[0]), str(comp_data[1]), str(comp_data[2])])
                data.append(comp_str)
                font_colors.append("green")
                font_sizes.append(12)
                font_weights.append("normal")
            coord_x = 0.05 + 0.5 * x_idx / total_lines
            coord_y = y_idx / players_in_line + (1 / players_in_line) / 2
            if x_idx == 3:
                coord_y = min(coord_y, 0.75)
                coord_y = max(coord_y, 0.25)
            if pos == 'right':
                coord_x = 1 - coord_x
                coord_y = 1 - coord_y
            rainbow_text(coord_x, coord_y, strings=data, font_colors=font_colors, ax=ax, font_sizes=font_sizes, font_weights=font_weights, ha='center', va='center')
    if coach:
        coord_x = 0.45
        if pos == 'right':
            coord_x = 1 - coord_x
        rainbow_text(coord_x, 0.075, strings=[coach], font_colors=["black"], ax=ax, font_sizes=["14"], font_weights=["extra bold"], ha='center', va='center')


In [ ]:
def print_referee(data):
    font_colors = ["red", "black", "blue", "black", "blue"]
    font_sizes = [14, 10, 10, 10, 10]
    font_weights = ['extra bold', 'black', 'normal', 'normal', 'normal']
    coord_x = 0.5
    coord_y = 0.98
    rainbow_text(coord_x, coord_y, strings=data, font_colors=font_colors, ax=ax, font_sizes=font_sizes, font_weights=font_weights, ha='center', va='center')

In [ ]:
def get_team(team_data, team_modulo):
    goalkeeper = team_data[0]
    team_lines = [int(line) for line in team_modulo.split('-')]
    data_lines = []
    data_lines.append([goalkeeper])
    idx = 1
    for line in team_lines:
        data_lines.append(team_data[idx:idx + line])
        idx += line
    return data_lines

In [ ]:
def get_team_data(team_data, team_id):
    team_names = [name for name, _ in team_data]
    team_numbers = [number for _, number in team_data]
    total_team_data = get_complete_data(team_names, lookup_date=match_date)
    for idx, (elem, number) in enumerate(zip(total_team_data, team_numbers)):
        elem['number'] = number
        elem['which_team'] = team_id
        if idx < 11:
            elem['is_bench'] = False
        else:
            elem['is_bench'] = True
    return total_team_data

In [ ]:
def draw_pitch_with_players(home_team_data, away_team_data, home_team_modulo, away_team_modulo, competition,
                            coach_home, coach_away, referees, match_date, output_directory):
    fig = plt.figure()
    fig.set_size_inches(1.5 * 11.7, 1.5 * 8.3)
    ax = fig.add_subplot(1,1,1)
    draw_pitch(ax)

    plt.axis('off')

    home_team = get_team(team_data=home_team_data, team_modulo=home_team_modulo)
    away_team = get_team(team_data=away_team_data, team_modulo=away_team_modulo)

    print_team(df=df, team=home_team, ax=ax, pos='left', comp=competition, coach=coach_home)
    print_team(df=df, team=away_team, ax=ax, pos='right', comp=competition, coach=coach_away)
    print_referee(referees)

    plt.ylim(0, 80)
    plt.xlim(0, 120)

    if not os.path.exists(output_directory):
        os.mkdir(output_directory)
    elif not os.path.isdir(output_directory):
        raise ValueError(f'{output_directory} is not a folder! Please double check it!')
    
    arrow_date = arrow.get(match_date, 'DD/MM/YYYY')
    date_str = arrow_date.format('YYYY-MM-DD')
    path = os.path.join(output_directory, f'{date_str}_{home_team_name}-{away_team_name}_{competition}.pdf')
    plt.savefig(path)
    plt.show()

In [ ]:
def draw_subs(df, home_team, away_team, competition, match_date, output_directory):
    fig1 = plt.figure()
    fig1.set_size_inches(1.5 * 11.7, 1.5 * 8.3)
    ax1 = fig1.add_subplot(1,1,1)

    plt.axis('off')   

    print_bench(df=df, team=home_team, ax=ax1, comp=competition, pos='left')
    print_bench(df=df, team=away_team, ax=ax1, comp=competition, pos='right')

    plt.ylim(0, 80)
    plt.xlim(0, 120)
    if not os.path.exists(output_directory):
        os.mkdir(output_directory)
    elif not os.path.isdir(output_directory):
        raise ValueError(f'{output_directory} is not a folder! Please double check it!')

    arrow_date = arrow.get(match_date, 'DD/MM/YYYY')
    date_str = arrow_date.format('YYYY-MM-DD')
    path = os.path.join(output_directory, f'{date_str}_{home_team_name}-{away_team_name}_{competition}_subs.pdf')
    plt.savefig(path)
    plt.show()

In [ ]:
def draw_table(df):
    keys = set()
    for idx, row in df.iterrows():
        for key in row['year_summary_data'].keys():
            keys.add(key)

    first_row = ['NomeCognome']
    for key in keys:
        first_row.extend(['Presenze', 'Gol', 'Assist', 'Gialli', 'DoppiGialli', 'Rossi', 'GolConcessi'])

    print('\t'.join(key.replace(' ', '') for key in keys))
    print('\t'.join(first_row))

    for idx, row in df.iterrows():
        player_row = [row['ref_url']]
        for key in keys:
            v1, v2, v3, v4, v5, v6 = row['year_summary_data'].get(key, ('-', '-', '-', '-', '-', '-'))
            if row['position'].lower() == 'goalkeeper':
                player_row.extend([v1, v2, '-', v3, v4, v5, v6])
            else:
                player_row.extend([v1, v2, v3, v4, v5, v6, '-'])
        print('\t'.join(player_row))

### Match Data

In [ ]:
match_date = '06/01/2021'
competition = 'Serie A'

home_team_name = 'milan'
home_team_modulo = '4-2-3-1'
home_coach = 'Stefano Pioli'

away_team_name = 'juventus'
away_team_modulo = '4-4-2'
away_coach = 'Andrea Pirlo'

referees = ["Arbitro: Massimi. Irrati", "Guardalinee: Costanzo, Longo", "4o uomo: Doveri", "VAR: Orsato", "AVAR: Mondin"]

output_directory = '../output/'

In [ ]:
home_team = [
    ('Gianluigi Donnarumma', 99),
    ('Theo Hernandez', 19),
    ('Alessio Romagnoli', 13),
    ('Simon Kjaer', 24),
    ('Diogo Dalot', 5),
    ('Franck Kessie', 79),
    ('Davide Calabria', 2),
    ('Jens Petter Hauge', 15),
    ('Hakan Calhanoglu', 10),
    ('Samu Castillejo', 7),
    ('Rafael Leão', 17),

    ('Ciprian Tatarusanu', 1),
    ('Antonio Donnarumma', 90),
    ('Andrea Conti', 14),
    ('Pierre Kalulu', 20),
    ('Mateo Musacchio', 22),
    ('Leo Duarte', 43),
    ('Marco Romano Frigerio', 97),
    ('Brahim Díaz', 21),
    ('Daniel Maldini', 27),
    ('Lorenzo Colombo', 29)
]

In [ ]:
away_team = [
    ('Wojciech Szczesny', 1),
    ('Gianluca Frabotta', 38),
    ('Matthijs de Ligt', 4),
    ('Leonardo Bonucci', 19),
    ('Danilo Luiz da Silva', 13),
    ('Aaron Ramsey', 8),
    ('Adrien Rabiot', 25),
    ('Rodrigo Bentancur', 30),
    ('Federico Chiesa', 22),
    ('Cristiano Ronaldo', 7),
    ('Paulo Dybala', 10),

    ('Gianluigi Buffon', 77),
    ('Carlo Pinsoglio', 31),
    ('Merih Demiral', 28),
    ('Giorgio Chiellini', 3),
    ('Arthur Melo', 5),
    ('Weston McKennie', 14),
    ('Alessandro Di Pardo', 36),
    ('Manolo Portanova', 39),
    ('Nicolò Fagioli', 41),
    ('Dejan Kulusevski', 44),
    ('Federico Bernardeschi', 33),
    ('Cosimo Marco Da Graca', 34),
]

### Get data in Pandas dataframe

In [ ]:
home_team_data = get_team_data(home_team, team_id=0)

In [ ]:
away_team_data = get_team_data(away_team, team_id=1)

In [ ]:
total_data = []
total_data.extend(home_team_data)
total_data.extend(away_team_data)

df = pd.DataFrame(total_data)

In [ ]:
df

### Plot data

In [ ]:
draw_pitch_with_players(home_team_data, away_team_data, home_team_modulo, away_team_modulo, competition,
                        home_coach, away_coach, referees, match_date, output_directory)

In [ ]:
draw_subs(df, home_team, away_team, competition, match_date, output_directory)

In [ ]:
draw_table(df)